In [1]:
import torch.nn as nn
import pandas as pd
import torch
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In [2]:
train_df = pd.read_csv("sign_mnist_train.csv")
valid_df = pd.read_csv("sign_mnist_valid.csv")

In [5]:
sample_df = train_df.head().copy()
sample_df.pop('label')
sample_x = sample_df.values
sample_x

array([[107, 118, 127, ..., 204, 203, 202],
       [155, 157, 156, ..., 103, 135, 149],
       [187, 188, 188, ..., 195, 194, 195],
       [211, 211, 212, ..., 222, 229, 163],
       [164, 167, 170, ..., 163, 164, 179]])

In [6]:
sample_x.shape

(5, 784)

In [7]:
img_height = 28
img_width = 28
img_chs = 1

sample_x = sample_x.reshape(-1, img_chs, img_height, img_width)
sample_x.shape

(5, 1, 28, 28)

#Create Dataset

In [17]:
class MyDataset(Dataset):
  def __init__(self, base_df):
    x_df = base_df.copy()
    y_df = x_df.pop('label')
    x_df = x_df.values / 255
    x_df = x_df.reshape(-1, img_chs, img_width, img_height)
    self.xs = torch.tensor(x_df).float().to(device)
    self.ys = torch.tensor(y_df).to(device)

  def __getitem__(self, idx):
    x = self.xs[idx]
    y = self.ys[idx]
    return x, y

  def __len__(self):
    return len(self.xs)

In [18]:
BATCH_SIZE = 32

train_data = MyDataset(train_df)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
train_N = len(train_loader.dataset)

valid_data = MyDataset(valid_df)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)
valid_N = len(valid_loader.dataset)

In [19]:
batch = next(iter(train_loader))
batch

[tensor([[[[0.6784, 0.6863, 0.6980,  ..., 0.7804, 0.7725, 0.7686],
           [0.6863, 0.6941, 0.7098,  ..., 0.7961, 0.7882, 0.7804],
           [0.6941, 0.7098, 0.7216,  ..., 0.8039, 0.8000, 0.7922],
           ...,
           [0.4980, 0.5020, 0.5059,  ..., 0.8471, 0.8549, 0.8392],
           [0.5373, 0.5373, 0.5373,  ..., 0.8902, 0.8510, 0.7843],
           [0.5255, 0.5294, 0.5333,  ..., 0.7020, 0.9412, 0.6118]]],
 
 
         [[[0.8078, 0.8078, 0.8157,  ..., 0.7647, 0.7569, 0.7529],
           [0.8196, 0.8196, 0.8196,  ..., 0.7647, 0.7647, 0.7569],
           [0.8235, 0.8314, 0.8275,  ..., 0.7647, 0.7647, 0.7647],
           ...,
           [0.7451, 0.3804, 0.3882,  ..., 0.8431, 0.8431, 0.8353],
           [0.4471, 0.3647, 0.3255,  ..., 0.8510, 0.8471, 0.8392],
           [0.3922, 0.3490, 0.3059,  ..., 0.8510, 0.8471, 0.8431]]],
 
 
         [[[0.5059, 0.5255, 0.5490,  ..., 0.6157, 0.6157, 0.6118],
           [0.5137, 0.5294, 0.5490,  ..., 0.6196, 0.6235, 0.6235],
           [0.5176

In [20]:
batch[0].shape

torch.Size([32, 1, 28, 28])

In [21]:
batch[1].shape

torch.Size([32])

#CNN Model

In [25]:
n_classes = 24
kernel_size = 3
flattened_img_size = 75*3*3

model = nn.Sequential(
    #First
    nn.Conv2d(img_chs, 25, kernel_size, stride=1, padding=1),
    nn.BatchNorm2d(25),
    nn.ReLU(),
    nn.MaxPool2d(2, stride = 2),
    #Second
    nn.Conv2d(25, 50, kernel_size, stride=1, padding=1),
    nn.BatchNorm2d(50),
    nn.ReLU(),
    nn.Dropout(.2),
    nn.MaxPool2d(2, stride = 2),
    #Third
    nn.Conv2d(50, 75, kernel_size, stride=1, padding=1),
    nn.BatchNorm2d(75),
    nn.ReLU(),
    nn.MaxPool2d(2, stride = 2),
    #Flatten
    nn.Flatten(),
    nn.Linear(flattened_img_size, 512),
    nn.Dropout(.3),
    nn.ReLU(),
    nn.Linear(512, n_classes)
)

In [26]:
model = torch.compile(model.to(device))
model

OptimizedModule(
  (_orig_mod): Sequential(
    (0): Conv2d(1, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(25, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(50, 75, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(75, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Flatten(start_dim=1, end_dim=-1)
    (14): Linear(in_features=675, out_features=512, bias=True)
    (15): Dropout

In [27]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

In [33]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

#Training

In [34]:
def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [37]:
def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [38]:
epochs = 20

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()

Epoch: 0
Train - Loss: 7.7375 Accuracy: 0.9978
Valid - Loss: 20.0500 Accuracy: 0.9743
Epoch: 1
Train - Loss: 1.9676 Accuracy: 0.9995
Valid - Loss: 18.2076 Accuracy: 0.9767
Epoch: 2
Train - Loss: 6.2344 Accuracy: 0.9978
Valid - Loss: 21.5801 Accuracy: 0.9757
Epoch: 3
Train - Loss: 0.7750 Accuracy: 0.9998
Valid - Loss: 59.4457 Accuracy: 0.9286
Epoch: 4
Train - Loss: 9.6023 Accuracy: 0.9969
Valid - Loss: 32.6562 Accuracy: 0.9658
Epoch: 5
Train - Loss: 7.3993 Accuracy: 0.9979
Valid - Loss: 34.5925 Accuracy: 0.9551
Epoch: 6
Train - Loss: 1.6282 Accuracy: 0.9995
Valid - Loss: 47.6087 Accuracy: 0.9596
Epoch: 7
Train - Loss: 2.0823 Accuracy: 0.9992
Valid - Loss: 35.3420 Accuracy: 0.9696
Epoch: 8
Train - Loss: 7.0001 Accuracy: 0.9978
Valid - Loss: 27.8267 Accuracy: 0.9776
Epoch: 9
Train - Loss: 2.4595 Accuracy: 0.9989
Valid - Loss: 23.1830 Accuracy: 0.9769
Epoch: 10
Train - Loss: 3.5046 Accuracy: 0.9987
Valid - Loss: 27.0667 Accuracy: 0.9755
Epoch: 11
Train - Loss: 0.1202 Accuracy: 1.0000
Valid

The training accuracy is very high, and the validation accuracy is also high as well.